In [69]:
from tqdm import tqdm
import numpy as np
import json
import os
import datetime
import pandas as pd
import sys
sys.path.append('/Users/jonat/OneDrive - University of Southern California/Documents/USC/Quant/Sports Betting/Sports-Betting')
from basketball_ref_scraper.constants import TEAM_TO_TEAM_ABBR
from basketball_ref_scraper.constants import CURR_NBA_TEAMS
from models.team_model import TeamStatistics

from models.dataload import DataLoad

player_metrics = ['DATE','MP','TS%','eFG%','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg','GAME_SCORE','BPM']
game_metrics = ["OPPONENT", "HOME", "STREAK"]
target_metrics =  ["POINTS_SCORED", "OPPONENT_POINTS", "OUTCOME"]
data = []
test_data = []
data.append(DataLoad(2017, player_metrics, game_metrics, target_metrics))
data.append(DataLoad(2018, player_metrics, game_metrics, target_metrics))
data.append(DataLoad(2019, player_metrics, game_metrics, target_metrics))
data.append(DataLoad(2020, player_metrics, game_metrics, target_metrics))
data.append(DataLoad(2021, player_metrics, game_metrics, target_metrics))
data.append(DataLoad(2022, player_metrics, game_metrics, target_metrics))
test_data.append(DataLoad(2023, player_metrics, game_metrics, target_metrics))

Getting data for ATL
Done
Getting data for BOS
Done
Getting data for BRK
Done
Getting data for CHO
Done
Getting data for CHI
Done
Getting data for CLE
Done
Getting data for DAL
Done
Getting data for DEN
Done
Getting data for DET
Done
Getting data for GSW
Done
Getting data for HOU
Done
Getting data for IND
Done
Getting data for LAC
Done
Getting data for LAL
Done
Getting data for MEM
Done
Getting data for MIA
Done
Getting data for MIL
Done
Getting data for MIN
Done
Getting data for NOP
Done
Getting data for NYK
Done
Getting data for OKC
Done
Getting data for ORL
Done
Getting data for PHI
Done
Getting data for PHO
Done
Getting data for POR
Done
Getting data for SAC
Done
Getting data for SAS
Done
Getting data for TOR
Done
Getting data for UTA
Done
Getting data for WAS
Done
Combining data for ATL
Done
Combining data for BOS
Done
Combining data for BRK
Done
Combining data for CHO
Done
Combining data for CHI
Done
Combining data for CLE
Done
Combining data for DAL
Done
Combining data for DEN
D

In [70]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [71]:
columns = ["HOME", "STREAK", 'DATE','MP','TS%','eFG%','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg','GAME_SCORE','BPM', "POINTS_SCORED", "OPPONENT_POINTS", "OUTCOME", 'SPREAD', 'OVER_UNDER']
data_columns = ["HOME", "STREAK", 'DATE','MP','TS%','eFG%','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','ORtg','DRtg','GAME_SCORE','BPM']
df = []

for data_year in data:
    for i in range(len(data_year.combined_team_sequence)):
        team_abbr = TEAM_TO_TEAM_ABBR[CURR_NBA_TEAMS[i].upper()]
        df.append(pd.DataFrame(data_year.combined_team_sequence[team_abbr], columns = columns)[5:])
for i in range(len(test_data[0].combined_team_sequence)):
    team_abbr = TEAM_TO_TEAM_ABBR[CURR_NBA_TEAMS[i].upper()]
    test_df = pd.DataFrame(test_data[0].combined_team_sequence[team_abbr], columns = columns)
# print(len(df))
drop_columns = ["POINTS_SCORED", "OPPONENT_POINTS", "OUTCOME", 'SPREAD', 'OVER_UNDER']
useless_columns = ["DATE", "MP"]

DEN_data = test_data[0].cumulative_sequence["DEN"][-1]
MIN_data = test_data[0].cumulative_sequence["MIN"][-1]
stat_diff = []
for j in range(len(DEN_data)):
    if j == 0:
        stat_diff.append(DEN_data[j])
    if j != 0:
        stat_diff.append(DEN_data[j] - MIN_data[j])
        
# from pov DEN:
DEN_MIN_game = list([1, -1] + stat_diff[2:])

LAC_data = test_data[0].cumulative_sequence["LAC"][-1]
PHO_data = test_data[0].cumulative_sequence["PHO"][-1]
stat_diff2 = []
for j in range(len(PHO_data)):
    if j == 0:
        stat_diff2.append(PHO_data[j])
    if j != 0:
        stat_diff2.append(PHO_data[j] - LAC_data[j])
        
# from pov DEN:
PHO_LAC_game = list([1, 3] + stat_diff2[2:])

print(DEN_MIN_game, PHO_LAC_game)

# df_data = []
# df_target = []

# for i in df:
#     df_data.append((i.drop(drop_columns, axis = 1)).drop(useless_columns, axis = 1))
#     df_target.append(i[drop_columns])

df = pd.concat(df, axis = 0)
    
df_data = df.drop(drop_columns, axis = 1)
test_df_data = test_df.drop(drop_columns, axis = 1)
df_data = df_data.drop(useless_columns, axis = 1)
test_df_data = test_df_data.drop(useless_columns, axis = 1)

df_target = df[drop_columns]
test_df_target = test_df[drop_columns]
df_data.head()

[1, -1, 0.2822773609719178, 0.3353544850053396, -4.6006540573831245, -6.043744676092842, -6.891520272775011, 4.484032083276674, 1.4542432787599182, -3.642207344173073, -4.14367519017955, -2.163506273853585, 31.114296815717807, 30.3407376289141, 0.8349623404070314, 11.180231644655407] [1, 3, -0.002605839566754753, -0.1221329859260436, -17.010668313943526, -23.46604842175796, -22.768565296904853, 45.311427940859716, -0.7101591182732676, -6.315536191945867, -22.16501996114718, 15.01528196320723, 58.24594654097416, 22.405036546133033, 18.24968340071291, 14.485711879326729]


,HOME,STREAK,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GAME_SCORE,BPM
5,1,1,-0.205584,-0.194145,6.686333,3.365161,4.356374,-7.258774,1.966919,5.658628,18.335060,-2.345893,-48.423100,-70.373407,-12.867742,-9.304122
6,0,2,-0.045985,-0.046203,-4.448211,-0.304517,-2.569402,-11.614260,2.348852,1.838945,12.922564,-7.232140,-48.656818,-34.559237,-14.194318,-8.677569
7,1,3,-0.050600,0.010067,-6.246434,-2.567943,-3.804559,2.877845,0.300177,1.434073,21.571100,3.379240,-62.203502,-42.631443,-5.088567,-8.209697
8,1,4,0.327429,0.280479,10.333707,1.340580,5.717899,-3.860826,0.337121,-0.520598,2.836669,-3.246964,65.847661,-43.304197,13.742536,9.022079
9,0,5,0.111300,0.034281,3.791911,4.846327,4.503322,3.693645,3.316298,-0.214529,13.919176,-4.165846,15.396661,-41.521267,1.277070,1.041620


In [130]:
X = df_data.values
y = df_target["SPREAD"].values
X_2023 = test_df_data.values
y_2023 = test_df_target["SPREAD"].values

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

10387 2597 10387 2597


In [8]:
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

torch.manual_seed(0)

model = Net(n_features=X.shape[1])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

n_epochs = 151
best_epoch = 0
best_acc = 0
best_model = model

for epoch in range(n_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        with torch.no_grad():
            outputs_test = model(X_test)
            loss_test = criterion(outputs_test, y_test.unsqueeze(1))
            acc_test = ((outputs_test > 0.5).float() == y_test.unsqueeze(1)).float().mean()
            if acc_test > best_acc:
                best_acc = acc_test
                best_epoch = epoch
                best_model = model
        print(f'Epoch {epoch}: train_loss={loss:.4f}, test_loss={loss_test:.4f}, test_acc={acc_test:.4f}')
print("Best epoch is {} with accuracy {}".format(best_epoch, best_acc))

Epoch 0: train_loss=0.8579, test_loss=0.8013, test_acc=0.4967
Epoch 1: train_loss=0.8061, test_loss=0.7715, test_acc=0.5040
Epoch 2: train_loss=0.7710, test_loss=0.7540, test_acc=0.5117
Epoch 3: train_loss=0.7492, test_loss=0.7438, test_acc=0.5125
Epoch 4: train_loss=0.7360, test_loss=0.7380, test_acc=0.5117
Epoch 5: train_loss=0.7281, test_loss=0.7341, test_acc=0.5114
Epoch 6: train_loss=0.7233, test_loss=0.7315, test_acc=0.5137
Epoch 7: train_loss=0.7203, test_loss=0.7298, test_acc=0.5137
Epoch 8: train_loss=0.7183, test_loss=0.7285, test_acc=0.5106
Epoch 9: train_loss=0.7171, test_loss=0.7274, test_acc=0.5087
Epoch 10: train_loss=0.7161, test_loss=0.7259, test_acc=0.5064
Epoch 11: train_loss=0.7145, test_loss=0.7236, test_acc=0.5044
Epoch 12: train_loss=0.7122, test_loss=0.7207, test_acc=0.5071
Epoch 13: train_loss=0.7092, test_loss=0.7176, test_acc=0.5125
Epoch 14: train_loss=0.7061, test_loss=0.7148, test_acc=0.5164
Epoch 15: train_loss=0.7032, test_loss=0.7124, test_acc=0.5210
Ep

In [15]:
output = best_model(torch.FloatTensor(X_2023))
probabilities = torch.sigmoid(output)

In [20]:
output = best_model(torch.FloatTensor(np.array(PHO_LAC_game)))
probabilities = torch.sigmoid(output)
print(output, probabilities)

tensor([0.7017], grad_fn=<SigmoidBackward0>) tensor([0.6686], grad_fn=<SigmoidBackward0>)


In [21]:
output = best_model(torch.FloatTensor(np.array(DEN_MIN_game)))
probabilities = torch.sigmoid(output)
print(output, probabilities)

tensor([0.6761], grad_fn=<SigmoidBackward0>) tensor([0.6629], grad_fn=<SigmoidBackward0>)


In [157]:
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

torch.manual_seed(0)

model = Net(n_features=X.shape[1])

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

n_epochs = 151
best_epoch = 0
best_acc = 0
best_model = model

for epoch in range(n_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        with torch.no_grad():
            outputs_test = model(X_test)
            loss_test = criterion(outputs_test, y_test.unsqueeze(1))
            acc_test = ((outputs_test > 0.5).float() == y_test.unsqueeze(1)).float().mean()
            if acc_test > best_acc:
                best_acc = acc_test
                best_epoch = epoch
                best_model = model
        print(f'Epoch {epoch}: train_loss={loss:.4f}, test_loss={loss_test:.4f}, test_acc={acc_test:.4f}')
print("Best epoch is {} with accuracy {}".format(best_epoch, best_acc))

Epoch 0: train_loss=216.2543, test_loss=206.0581, test_acc=0.0039
Epoch 1: train_loss=213.4375, test_loss=204.9152, test_acc=0.0054
Epoch 2: train_loss=211.4592, test_loss=204.3984, test_acc=0.0062
Epoch 3: train_loss=210.2212, test_loss=204.2798, test_acc=0.0089
Epoch 4: train_loss=209.5303, test_loss=204.3492, test_acc=0.0108
Epoch 5: train_loss=209.1885, test_loss=204.4021, test_acc=0.0116
Epoch 6: train_loss=209.0229, test_loss=204.3424, test_acc=0.0119
Epoch 7: train_loss=208.8942, test_loss=204.1630, test_acc=0.0123
Epoch 8: train_loss=208.7446, test_loss=203.8851, test_acc=0.0119
Epoch 9: train_loss=208.5582, test_loss=203.5613, test_acc=0.0119
Epoch 10: train_loss=208.3595, test_loss=203.2315, test_acc=0.0123
Epoch 11: train_loss=208.1576, test_loss=202.9297, test_acc=0.0119
Epoch 12: train_loss=207.9521, test_loss=202.6586, test_acc=0.0112
Epoch 13: train_loss=207.7433, test_loss=202.4252, test_acc=0.0112
Epoch 14: train_loss=207.5321, test_loss=202.2334, test_acc=0.0108
Epoch

In [160]:
output = best_model(torch.FloatTensor(X_2023))
print(output[5:10], y_2023[5:10])

tensor([[ 1.2190],
        [-1.2265],
        [-3.8108],
        [-0.0108],
        [ 3.2417],
        [-2.7134],
        [ 0.4882],
        [ 1.9772],
        [ 3.6756],
        [-0.8403],
        [-0.8552],
        [-3.1762],
        [-1.9820],
        [ 3.2180],
        [-7.6127],
        [ 0.2589],
        [-1.1646],
        [-1.5260],
        [-2.1488],
        [-1.8553],
        [ 0.9946],
        [ 3.9438],
        [-1.0880],
        [-1.8939],
        [-4.8851],
        [ 0.7787],
        [-2.6713],
        [ 0.5401],
        [-4.4126],
        [-3.2340],
        [-4.3524],
        [-3.3085],
        [-5.9498],
        [-1.6577],
        [-0.6268],
        [-2.3730],
        [ 0.2438],
        [-3.3937],
        [-4.3768],
        [-9.1599],
        [-3.2987],
        [-2.4811],
        [ 3.9763],
        [-2.1942],
        [ 3.8984],
        [-0.0240],
        [-5.7785],
        [-0.5853],
        [ 3.4827],
        [-0.3194],
        [ 4.7877],
        [-4.5284],
        [-3.

In [159]:
output = best_model(torch.FloatTensor(np.array(PHO_LAC_game)))
probabilities = torch.sigmoid(output)
print(output, probabilities)

tensor([3.5613], grad_fn=<AddBackward0>) tensor([0.9724], grad_fn=<SigmoidBackward0>)


In [136]:
output = best_model(torch.FloatTensor(np.array(DEN_MIN_game)))
probabilities = torch.sigmoid(output)
print(output, probabilities)

tensor([4.5003], grad_fn=<AddBackward0>) tensor([0.9890], grad_fn=<SigmoidBackward0>)


In [146]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# Train the decision tree model
model = DecisionTreeRegressor()
model.fit(X_2023, y_2023)

print(type(X_test), X_test.shape)

# Make predictions on the test data
y_pred = model.predict(X_test)
print(y_pred)

# Calculate the accuracy of the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

<class 'torch.Tensor'> torch.Size([2597, 16])
[  1.  -9.  23. ... -13.  23.   8.]
Root Mean Squared Error: 21.652638053915684


In [152]:
game_input = np.vstack([np.array(DEN_MIN_game), np.array(PHO_LAC_game)])
print(game_input.shape)
output = model.predict(game_input)
print(output)

(2, 16)
[23. -9.]


In [156]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score


model = LinearRegression()

model.fit(X, y)

y_train_pred = model.predict(X)

train_acc = mean_absolute_error(y, y_train_pred)

y_test_pred = model.predict(X_2023)
# confidence = model.predict_proba(X_2023)

print(y_test_pred)
print(y_2023)
# print(confidence)

test_acc = mean_absolute_error(y_2023, y_test_pred)


print('Training Acc: {:.2f}'.format(train_acc))
print('Test Acc: {:.2f}'.format(test_acc))

[-0.39721284  0.34988462 -0.9831657   3.62576508 -2.55308091 -0.84370713
 -1.10295159  0.12539137  0.02076833 -3.34325176  0.97430443 -2.11012666
 -0.70002931 -0.98609867 -2.10743864 -0.93134348  1.66396496 -1.86961883
 -2.23547824 -1.49124591  1.43721249 -0.58254728 -0.27996454 -3.43724881
 -2.60097425 -5.35456629 -2.70283061 -2.76559982 -5.4277075  -4.13679882
 -6.42230968 -2.14831959 -2.54596746 -2.43603486 -0.35688797 -1.15941195
  4.43326929 -1.94524876 -4.25520327 -3.19452642 -2.72709317 -0.35705093
 -0.2756263  -3.11648762  0.31992494  3.95230368 -2.30522142  0.18287619
  5.3084646  -0.74435689 -1.32897688 -2.95261849  1.36970632 -2.45749585
 -2.87790451 -0.20195933  1.75683835 -0.13339459 -1.60231346  0.88150503
 -0.91166573 -2.05220227 -4.2487059   3.75673596  0.31149378 -0.07204257
 -2.67932612  4.08368855 -2.48006198 -3.12019933  1.84212553 -3.79373684
  3.68040899 -2.53732696 -0.06553998  2.7770024  -0.85044174 -4.67340617
 -1.00071391]
[  2 -10  21 -10 -18  -7  10 -42  -6 